In [1]:
import numpy as np
import tensorflow as tf
import pickle
import random
from matplotlib import pyplot as plt
import math
from PIL import Image
from matplotlib.pyplot import imshow
import h5py
import gc
import sys

In [2]:
hdf_file = 'datasets/pickles/box/SVHN_multi_box.hdf5'

hdf = h5py.File(hdf_file,'r')
svhn_train_box_dataset = hdf['train_images'][:]
svhn_train_box_labels = hdf['train_labels'][:]
svhn_test_box_dataset = hdf['test_images'][:]
svhn_test_box_labels = hdf['test_labels'][:]
svhn_valid_box_dataset = hdf['valid_images'][:]
svhn_valid_box_labels = hdf['valid_labels'][:]
            
hdf.close()    

print(svhn_train_box_dataset.shape, svhn_train_box_labels.shape)
print(svhn_test_box_dataset.shape, svhn_test_box_labels.shape)
print(svhn_valid_box_dataset.shape, svhn_valid_box_labels.shape)

(230069, 32, 96, 1) (230069, 6)
(13068, 32, 96, 1) (13068, 6)
(5684, 32, 96, 1) (5684, 6)


In [3]:
svhn_train_box_dataset = svhn_train_box_dataset.astype(np.float32)
svhn_test_box_dataset = svhn_test_box_dataset.astype(np.float32)
svhn_valid_box_dataset = svhn_valid_box_dataset.astype(np.float32)

svhn_train_box_labels = svhn_train_box_labels.astype(np.int32)
svhn_test_box_labels = svhn_test_box_labels.astype(np.int32)
svhn_valid_box_labels = svhn_valid_box_labels.astype(np.int32)

In [2]:
hdf_file = 'datasets/pickles/SVHN_multi.hdf5'

hdf = h5py.File(hdf_file,'r')
svhn_test_dataset = hdf['test_images'][:]
svhn_test_labels = hdf['test_labels'][:]
svhn_train_dataset = hdf['train_images'][:]
svhn_train_labels = hdf['train_labels'][:]
svhn_valid_dataset = hdf['valid_images'][:]
svhn_valid_labels = hdf['valid_labels'][:]

hdf.close()    

print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

(230255, 32, 96, 1) (230255, 6)
(13068, 32, 96, 1) (13068, 6)
(5500, 32, 96, 1) (5500, 6)


In [20]:
gc.collect()

0

In [3]:
%whos

Variable             Type        Data/Info
------------------------------------------
Image                module      <module 'PIL.Image' from <...>e-packages/PIL/Image.py'>
gc                   module      <module 'gc' (built-in)>
h5py                 module      <module 'h5py' from '/hom<...>ckages/h5py/__init__.py'>
hdf                  File        <Closed HDF5 file>
hdf_file             str         datasets/pickles/SVHN_multi.hdf5
imshow               function    <function imshow at 0x7fb69e53f730>
math                 module      <module 'math' from '/hom<...>35m-x86_64-linux-gnu.so'>
np                   module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pickle               module      <module 'pickle' from '/h<...>lib/python3.5/pickle.py'>
plt                  module      <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
random               module      <module 'random' from '/h<...>lib/python3.5/random.py'>
svhn_test_dataset    ndarray     13068x32x96x1

In [22]:
del svhn_test_box_dataset, svhn_test_box_labels, svhn_train_box_dataset, svhn_train_box_labels, svhn_valid_box_dataset, svhn_valid_box_labels

In [25]:
% whos

Variable             Type         Data/Info
-------------------------------------------
B1                   Variable     Tensor("Variable_1/read:0<...>hape=(6,), dtype=float32)
B2                   Variable     Tensor("Variable_3/read:0<...>ape=(12,), dtype=float32)
B3                   Variable     Tensor("Variable_5/read:0<...>ape=(24,), dtype=float32)
B4                   Variable     Tensor("Variable_17/read:<...>pe=(200,), dtype=float32)
B5_1                 Variable     Tensor("Variable_7/read:0<...>ape=(11,), dtype=float32)
B5_2                 Variable     Tensor("Variable_9/read:0<...>ape=(11,), dtype=float32)
B5_3                 Variable     Tensor("Variable_11/read:<...>ape=(11,), dtype=float32)
B5_4                 Variable     Tensor("Variable_13/read:<...>ape=(11,), dtype=float32)
B5_5                 Variable     Tensor("Variable_15/read:<...>ape=(11,), dtype=float32)
HEIGHT               int          32
Image                module       <module 'PIL.Image' from <...>e

In [4]:
sys.getsizeof(pickle)

80

In [23]:
gc.collect()

0

In [5]:
svhn_train_dataset = svhn_train_dataset.astype(np.float32)
svhn_test_dataset = svhn_test_dataset.astype(np.float32)
svhn_valid_dataset = svhn_valid_dataset.astype(np.float32)

svhn_train_labels = svhn_train_labels.astype(np.int32)
svhn_test_labels = svhn_test_labels.astype(np.int32)
svhn_valid_labels = svhn_valid_labels.astype(np.int32)

In [32]:
print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

(230255, 32, 96, 1) (230255, 6)
(13068, 32, 96, 1) (13068, 6)
(5500, 32, 96, 1) (5500, 6)


In [10]:
svhn_train_dataset, svhn_train_labels = randomize_dataset(svhn_train_dataset, svhn_train_labels)
svhn_test_dataset, svhn_test_labels = randomize_dataset(svhn_test_dataset, svhn_test_labels)
svhn_valid_dataset, svhn_valid_labels = randomize_dataset(svhn_valid_dataset, svhn_valid_labels)

In [11]:
svhn_train_dataset_1 = svhn_train_dataset[0:125000]
svhn_train_labels_1 = svhn_train_labels[0:125000]

print(svhn_train_dataset_1.shape, svhn_train_labels_1.shape)
print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

(125000, 32, 96, 1) (125000, 6)
(230255, 32, 96, 1) (230255, 6)
(13068, 32, 96, 1) (13068, 6)
(5500, 32, 96, 1) (5500, 6)


In [4]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [5]:
def randomize_dataset(images, labels):
    shuffle = list(zip(images, labels))
    np.random.shuffle(shuffle)
    i, l = zip(*shuffle)
    i, l = np.asarray(i), np.asarray(l)
    return i, l

In [6]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [7]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3
    
    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 5 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 6     # First Conv Layer with depth 6
    L = 12     # Second Conv Layer with depth 12
    M = 24    # Third Conv layer with depth 24
    N = 200   # Fourth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1), name="W1")    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]), name="B1")
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1), name="W2")
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]), name="B2")
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1), name="W3")
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]), name="B3")
    
    W5_1 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_1")
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_1")
    
    W5_2 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_2")
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_2")
    
    W5_3 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_3")
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_3")
    
    W5_4 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_4")
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_4")
    
    W5_5 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_5")
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_5")
    
    # Model
    stride = 1  # output is 32x96
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 16x48
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 8x24
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1), name="W4")
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]), name="B4")

    Y4 = tf.sigmoid(tf.matmul(YY, W4) + B4)
    YY4 = tf.nn.dropout(Y4, pkeep)
    
    Ylogits_1 = tf.matmul(YY4, W5_1) + B5_1
    Ylogits_2 = tf.matmul(YY4, W5_2) + B5_2
    Ylogits_3 = tf.matmul(YY4, W5_3) + B5_3
    Ylogits_4 = tf.matmul(YY4, W5_4) + B5_4
    Ylogits_5 = tf.matmul(YY4, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
   
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    W_s = tf.pack([tf.reduce_max(tf.abs(W1)),tf.reduce_max(tf.abs(W2)),tf.reduce_max(tf.abs(W3)),tf.reduce_max(tf.abs(W4))])
    b_s = tf.pack([tf.reduce_max(tf.abs(B1)),tf.reduce_max(tf.abs(B2)),tf.reduce_max(tf.abs(B3)),tf.reduce_max(tf.abs(B4))])
    
    var_list = [W1, B1, W2, B2, W3, B3, W4, B4, W5_1, B5_1, W5_2, B5_2, W5_3, B5_3, W5_4, B5_4, W5_5, B5_5]
    model_saver = tf.train.Saver(var_list)

In [ ]:
train_data = svhn_train_box_dataset
label_data = svhn_train_box_labels
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 20001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001

        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
        _, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/box/CNN_SVHN_Box.ckpt")
    print("Model saved in file: %s" % save_path)

train :  (230069, 32, 96, 1)   test :  (230069, 6)
Initialized
W :  [ 0.19758295  0.19993503  0.19987057  0.19999866]
b :  [ 0.1  0.1  0.1  0.1]
Loss at step 0: 13.472513
Minibatch accuracy: 7.2%
Learning rate :  0.0005
    
W :  [ 0.26501966  0.30211484  0.27566063  0.29679999]
b :  [ 0.14650765  0.11654811  0.10576747  0.11026224]
Loss at step 500: 5.076310
Minibatch accuracy: 68.3%
Learning rate :  0.00046193496721438383
    


In [ ]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/box/CNN_SVHN_Box.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    batch = 1000
    
    test_acc = list()
    print('-------TEST--------')
    test_no = int(svhn_test_box_labels.shape[0] / batch)
    for i in range(test_no - 1):
        data = svhn_test_box_dataset[i*t_batch:(i+1)*batch]
        labels = svhn_test_box_labels[i*t_batch:(i+1)*batch]
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        accuracy = acc(predictions, labels[:,1:6])
        test_acc.append(accuracy)
        
        print('Test-Accuracy', ' i : ', i)
        print('Test accuracy: ', accuracy)
        print('       ')
    
    valid_acc = list()
    print('-----VALIDIDATION------')
    valid_no = int(svhn_valid_box_labels.shape[0] /  batch)
    for i in range(valid_no - 1):
        data = svhn_valid_box_dataset[i*t_batch:(i+1)*batch]
        labels = svhn_valid_box_labels[i*t_batch:(i+1)*batch]
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        accuracy = acc(predictions, labels[:,1:6])
        valid_acc.append(accuracy)
        
        print('Valid-Accuracy', ' i : ', i)
        print('Valid accuracy: ', accuracy)
        print('        ')
        
        
    test_avg = mean(test_acc)
    valid_avg = mean(valid_acc)
    
    print('-----  FINAL  ------')
    print('Final Test Set Accuracy : ',"%.2f" % test_avg)
    print('Final Validation Set Accuracy : ',"%.2f" % valid_avg)
        
#     _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_box_dataset, Y_ : svhn_valid_box_labels, pkeep : 1.0, alpha : 0.002})
#     print('Validation accuracy: ', acc(predictions, svhn_valid_box_labels[:,1:6]))


In [22]:
train_data = svhn_train_dataset_1
label_data = svhn_train_labels_1
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 3001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001
        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
        _, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)

        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/multi/CNN_SVHN_New.ckpt")
    print("Model saved in file: %s" % save_path)

train :  (125000, 32, 96, 1)   test :  (125000, 6)
Initialized
W :  [ 0.1957577   0.19950426  0.19999866  0.19999957]
b :  [ 0.1  0.1  0.1  0.1]
Loss at step 0: 15.812104
Minibatch accuracy: 3.4%
Learning rate :  0.0005
    
W :  [ 0.20176931  0.20336588  0.21233326  0.21602039]
b :  [ 0.09924942  0.11802243  0.11003009  0.11306362]
Loss at step 500: 6.913124
Minibatch accuracy: 53.9%
Learning rate :  0.00046193496721438383
    
W :  [ 0.20172596  0.20614482  0.21922968  0.232022  ]
b :  [ 0.10019359  0.12345539  0.11821508  0.1157744 ]
Loss at step 1000: 6.926769
Minibatch accuracy: 54.4%
Learning rate :  0.00042749230123119273
    
W :  [ 0.20099458  0.20611057  0.22664881  0.24046591]
b :  [ 0.10177581  0.12987101  0.11949582  0.11835108]
Loss at step 1500: 6.856278
Minibatch accuracy: 53.8%
Learning rate :  0.00039632728827268716
    
W :  [ 0.19851771  0.20685558  0.23405121  0.24046807]
b :  [ 0.10594938  0.12891285  0.12108289  0.11941691]
Loss at step 2000: 6.854865
Minibatch a

In [25]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/multi/CNN_SVHN_New.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    print(session.run(W1))

Model restored.
Initialized
[[[[ 0.06091791  0.0327538  -0.19416073 -0.01676372  0.02233364 -0.02901404]]

  [[-0.04115827 -0.00266697  0.10546505  0.03128034 -0.02302971  0.03785209]]

  [[ 0.02321358 -0.08577943 -0.07981791  0.19747455 -0.06205473  0.06701094]]

  [[-0.02935915 -0.09522378  0.05649186  0.08751354 -0.05960583 -0.07666398]]

  [[-0.14650303  0.17145529 -0.00110248  0.04165018  0.01372952  0.02088568]]

  [[ 0.01313367  0.00348472 -0.12836632  0.01750273 -0.06433355 -0.1076489 ]]]


 [[[ 0.01552692  0.19806165  0.01933289 -0.14826033 -0.04317336 -0.09452876]]

  [[ 0.04372241 -0.17828043  0.10888292  0.01148719 -0.05250533 -0.02962187]]

  [[-0.00225709 -0.13161032 -0.05115749  0.03142713  0.173245   -0.01677104]]

  [[-0.15817691 -0.09034646 -0.07150197 -0.16240661  0.06633522  0.05496876]]

  [[ 0.01151476 -0.12420844  0.02609648  0.12235687  0.0465115  -0.04505098]]

  [[ 0.0315448   0.05198722 -0.04371455  0.01141434  0.14327204  0.00390389]]]


 [[[ 0.03091958  0.0

In [24]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/box/CNN_SVHNox_.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    print(session.run([W1]))

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for saved_models/box/CNN_SVHNox_.ckpt
	 [[Node: save/RestoreV2_17 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_17/tensor_names, save/RestoreV2_17/shape_and_slices)]]
	 [[Node: save/RestoreV2_12/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_140_save/RestoreV2_12", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_17', defined at:
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python

In [ ]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/multi/CNN_SVHN_New.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_test_dataset[:500], Y_ : svhn_test_labels[:500], pkeep : 1.0, alpha : 0.002})
    print('Test accuracy: ', acc(predictions, svhn_test_labels[:500,1:6]))
    
    del l, predictions
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_dataset, Y_ : svhn_valid_labels, pkeep : 1.0, alpha : 0.002})
    print('Validation accuracy: ', acc(predictions, svhn_valid_labels[:,1:6]))

In [13]:
train_data = svhn_train_dataset_1
label_data = svhn_train_labels_1
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 20001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/box/CNN_SVHN_Box.ckpt")
    print("Model restored.") 
#     tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001
        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
        _, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/combined/CNN_SVHN_Mix.ckpt")
    print("Model saved in file: %s" % save_path)

train :  (125000, 32, 96, 1)   test :  (125000, 6)
Model restored.
Initialized
W :  [ 0.41171086  0.69925141  0.47237745  0.7207135 ]
b :  [ 0.53505075  0.31870961  0.45383301  0.12280813]
Loss at step 0: 54.654106
Minibatch accuracy: 16.9%
Learning rate :  0.0005
    
W :  [ 0.42643231  0.70004612  0.46439368  0.71949875]
b :  [ 0.53378701  0.31939578  0.45481381  0.12296873]
Loss at step 500: 8.034361
Minibatch accuracy: 52.0%
Learning rate :  0.00046193496721438383
    


KeyboardInterrupt: 